In [1]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pipeline import Dataset

In [2]:
dataset = Dataset(splits=(0.8, 0.2))
dataset.generate_datasplit()

C:\Users\Oliver\PycharmProjects\BigDataProject_IMDB\pipeline\_dataset.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self._complete_df["numVotes"].fillna(0, inplace=True)
C:\Users\Oliver\PycharmProjects\BigDataProject_IMDB\pipeline\_dataset.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always be

In [3]:
train = dataset.train_df[dataset.train_df.columns.drop(["primaryTitle", "originalTitle"])]
train_X, train_y = train[train.columns.drop('label')], train["label"]

test = dataset.test_df[dataset.train_df.columns.drop(["primaryTitle", "originalTitle"])]
test_X, test_y = test[test.columns.drop('label')], test["label"]

In [6]:
clf = RandomForestClassifier(max_depth=2, random_state=42)

# Define the parameter grid to search
param_grid = {
    'max_depth': [2, 4, 6, 8],
    'n_estimators': [50, 100, 200, 300]
}


# Initialize GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Fit the model with GridSearchCV
grid_search.fit(train_X, train_y)

# Get the best parameters and estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Print the best parameters
print("Best parameters:", best_params)

# Use the best estimator to make predictions
y_pred = best_estimator.predict(test_X)

# Measure accuracy
accuracy = accuracy_score(test_y, y_pred)
print("Accuracy:", accuracy)

# Perform K-fold cross-validation and print the mean accuracy
cv_accuracy = cross_val_score(best_estimator, train_X, train_y, cv=5, scoring='accuracy')
print("Cross-validated Accuracy:", cv_accuracy.mean())

Best parameters: {'max_depth': 6, 'n_estimators': 300}
Accuracy: 0.9904306220095693
Cross-validated Accuracy: 0.9853932797870824


### Overfitting a bit hehe 

## Prepare CSV files for kaggle

Asuming that the format will be index + prediction

In [198]:
validation = get_scores_val_test(validation)
test = get_scores_val_test(test)

In [199]:
validation["label"] = best_estimator.predict(validation[feature_cols])
test["label"] = best_estimator.predict(test[feature_cols])

validation["label"].to_csv("val_result.csv")
test["label"].to_csv("test_result.csv")